In [1]:
import pandas as pd
import numpy as np
import math
import glob

In [11]:
def getListBack(df, item, trial):
    trial = df[item][trial]
    trial = trial.replace('[', '')
    trial = trial.replace(']', '')
    trial = trial.split(',')
    trial = [float(i) for i in trial]
    return trial

def windowMSE(df, participant, frameRate, all_mouse_x, all_mouse_y, all_target_x, all_target_y, windowlist):
    print('Starting MSE calculation...')
    for window in windowlist:
        standard_error = [] #Make list with standard error
        for i in range(0,len(all_mouse_x)):
            if type(all_target_y[i]) == str:
                standard_error.append(all_target_y[i])
            else:
                p = [all_mouse_x[i], all_mouse_y[i]]
                q = [all_target_x[i], all_target_y[i]]
                standard_error.append(math.dist(p, q))

        all_error = 0
        for i, e in enumerate(standard_error):
            if type(e) == str:
                loop = 1
                count = 0
                dif_count = []
                while count != int(window*frameRate): #30 sec
                    cur = standard_error[i-int(3*frameRate)-loop]
                    if type(cur) == str:
                        loop += 1
                    else:
                        all_error += cur
                        loop += 1
                        count += 1 
                        if all_mouse_x[i-int(3*frameRate)-loop] not in dif_count:
                            dif_count.append(all_mouse_x[i-int(3*frameRate)-loop])
                
                if len(set(dif_count)) > frameRate:
                    result.loc[len(result)] = {'Participant': participant, 'Group': df['group'][0], 'window': window, 'report': 'probe', 'MW': e, 'MSD': all_error/int(window*frameRate)}
                else:
                    print('Did not move')
                all_error = 0

def probeCaught(current_data, participant):
    df = current_data
    frameRate = current_data['frameRate'][0]
    
    all_mouse_x = [0.0] * int(frameRate*35) #35 = 32s trial + 3s delay
    all_mouse_y = [0.0] * int(frameRate*35)
    all_target_x = [0.0] * int(frameRate*35)
    all_target_y = [0.0] * int(frameRate*35)
    
    print('Loading trials...')    
    for cur_trial in range (1,55): #Loop over trials
        if df['mouse_2.x'][cur_trial] == '':
            continue        
        
        mouse_x = getListBack(df, 'mouse_2.x', cur_trial) #Get mouse x, y  positions and t
        mouse_y = getListBack(df, 'mouse_2.y', cur_trial)
        mouse_t = getListBack(df, 'mouse_2.time', cur_trial)
        
        if df['key_resp_2.rt'][cur_trial] == '' and len(mouse_x) > 32: #Get cuurent frameRate (this can change)
            frameRate = len(mouse_x)/32
            
        t = [] #Get time
        for time in range(0, int(frameRate*32)):
            t.append(time/int(frameRate))
        
        if df['key_resp_2.rt'][cur_trial] != '': #Is there a probe in this trial?
            print("Trail {}: mouse(x,y,t) = ({},{},{})".format(cur_trial, len(mouse_x), len(mouse_y), len(t)))
            probe_times = getListBack(df, 'key_resp_2.rt', cur_trial) #Get probe time
            probe_time = probe_times[-1]
            probe_answer = df['key_resp_2.keys'][cur_trial][2]
            
            probe_index = 0
            for index, time in enumerate(mouse_t): #Find index of probe
                if time > probe_time and probe_index == 0:
                    probe_index = index
                    
            dif = len(mouse_x)-(int(frameRate*32)) #length of answer
            if len(probe_times) > 1:
                del mouse_x[probe_index-dif:probe_index] #Remove thinking time before probe
                del mouse_y[probe_index-dif:probe_index]
                mouse_x = mouse_x[int(-1*frameRate*30)-1:] #Remove first 2 seconds of trial
                mouse_y = mouse_y[int(-1*frameRate*30)-1:]
                t = t[int(-1*frameRate*30)-1:]
                
            elif probe_time > 2 and probe_index-dif > int(frameRate*2):
                mouse_x.insert(probe_index, probe_answer)
                mouse_y.insert(probe_index, probe_answer)
                t.insert(probe_index-dif, probe_answer)        
                
                del mouse_x[probe_index-dif:probe_index] #Remove thinking time before probe
                del mouse_y[probe_index-dif:probe_index]
                
                mouse_x = mouse_x[int(-1*frameRate*30)-1:] #Remove first 2 seconds of trial
                mouse_y = mouse_y[int(-1*frameRate*30)-1:]
                t = t[int(-1*frameRate*30)-1:]
                    
            else: #Probe in first 2 seconds
                mouse_x = mouse_x[int(-1*frameRate*30):] #Remove first 2 seconds of trial
                mouse_y = mouse_y[int(-1*frameRate*30):]
                t = t[int(-1*frameRate*30):]
                
                mouse_x.insert(0, probe_answer) #Insert probe
                mouse_y.insert(0, probe_answer)
                t.insert(0, probe_answer) 
                
        else: #Only remove first 2 seconds from trail
            mouse_x = mouse_x[int(-1*frameRate*30):]
            mouse_y = mouse_y[int(-1*frameRate*30):]
            t = t[int(-1*frameRate*30):]
        
        #if not len(mouse_x) == len(mouse_y) == len(t):
        #    t = t[:len(mouse_x)]          
        
        target_x = [] #Get target positions
        target_y = []
        for pt in t:
            if type(pt) == str:
                target_x.append(pt)
                target_y.append(pt)
            else:
                target_x.append(-0.8+(pt-2)/18.6)
                target_y.append((df['ab0'][cur_trial]+
                                 df['a1'][cur_trial]*math.sin(1*pt/5)+df['b1'][cur_trial]*math.cos(1*pt/5)+
                                 df['a2'][cur_trial]*math.sin(2*pt/5)+df['b2'][cur_trial]*math.cos(2*pt/5)+
                                 df['a3'][cur_trial]*math.sin(3*pt/5)+df['b3'][cur_trial]*math.cos(3*pt/5)+
                                 df['a4'][cur_trial]*math.sin(4*pt/5)+df['b4'][cur_trial]*math.cos(4*pt/5)+
                                 df['a5'][cur_trial]*math.sin(5*pt/5)+df['b5'][cur_trial]*math.cos(5*pt/5)+
                                 df['a6'][cur_trial]*math.sin(6*pt/5)+df['b6'][cur_trial]*math.cos(6*pt/5))/50)

        all_mouse_x = all_mouse_x + mouse_x #Make long list with all trials
        all_mouse_y = all_mouse_y + mouse_y
        all_target_x = all_target_x + target_x
        all_target_y = all_target_y + target_y
    
    windowMSE(df, participant, frameRate, all_mouse_x, all_mouse_y, all_target_x, all_target_y, [5,10,15,20,25,30])

In [12]:
# Create an empty DataFrame
result = pd.DataFrame(columns = ['Participant', 'Group', 'window', 'report', 'MW', 'MSD'])

In [13]:
for file_name in glob.glob(r"C:\Users\Robyn\Downloads\!experiment\data\SONA"+'\*.csv'):
    participant_nr = file_name[47:52]
    print('Participant: ', participant_nr)
    data = pd.read_csv('{}'.format(file_name))
    data = data.replace(np.nan, '')
    probeCaught(data, participant_nr)
    print('Done. Current results =', len(result), '\n')

Participant:  11003
Loading trials...
Trail 3: mouse(x,y,t) = (2896,2896,1920)
Trail 9: mouse(x,y,t) = (2135,2135,1921)
Trail 13: mouse(x,y,t) = (2092,2092,1921)
Trail 14: mouse(x,y,t) = (2086,2086,1921)
Trail 20: mouse(x,y,t) = (2013,2013,1921)
Trail 22: mouse(x,y,t) = (2050,2050,1921)
Starting MSE calculation...
Done. Current results = 36 

Participant:  13604
Loading trials...
Trail 4: mouse(x,y,t) = (2254,2254,1921)
Trail 5: mouse(x,y,t) = (2177,2177,1921)
Trail 10: mouse(x,y,t) = (2147,2147,1921)
Trail 13: mouse(x,y,t) = (2140,2140,1921)
Trail 15: mouse(x,y,t) = (2066,2066,1922)
Trail 21: mouse(x,y,t) = (2044,2044,1921)
Starting MSE calculation...
Done. Current results = 72 

Participant:  15496
Loading trials...
Trail 35: mouse(x,y,t) = (2275,2275,1922)
Trail 37: mouse(x,y,t) = (2044,2044,1921)
Trail 39: mouse(x,y,t) = (2103,2103,1921)
Trail 43: mouse(x,y,t) = (2057,2057,1921)
Trail 44: mouse(x,y,t) = (2028,2028,1921)
Trail 50: mouse(x,y,t) = (2053,2053,1921)
Starting MSE calcula

Trail 15: mouse(x,y,t) = (2076,2076,1918)
Trail 21: mouse(x,y,t) = (5031,5031,1919)
Starting MSE calculation...
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Did not move
Done. Current results = 753 

Participant:  49612
Loading trials...
Trail 32: mouse(x,y,t) = (9045,9045,7630)
Trail 33: mouse(x,y,t) = (9139,9139,7630)
Trail 39: mouse(x,y,t) = (10457,10457,7623)
Trail 41: mouse(x,y,t) = (8195,8195,7602)
Trail 47: mouse(x,y,t) = (8876,8876,7650)
Trail 51: mouse(x,y,t) = (11251,11251,7508)
Starting MSE calculation...
Did not move
Done. Current results = 788 

Participant:  50121
Loading trials...
Trail 35: mouse(x,y,t) = (1633,1633,960)
Trail 38: mouse(x,y,t) = (1047,1047,960)
Trail 43: mouse(x,y,t) = (1033,1033,960)
Trail 46: mouse(x,y,t) = (1012,1012,960)
Trail 51: mouse(x,y,t) = (1203,1203,960)
Trail 52: mouse(x,y,t) = (1040,1040,960)
Starting MSE calculation...
Did not move

Trail 10: mouse(x,y,t) = (3980,3980,1921)
Trail 12: mouse(x,y,t) = (2101,2101,1921)
Trail 15: mouse(x,y,t) = (2077,2077,1921)
Trail 16: mouse(x,y,t) = (2344,2344,1921)
Trail 22: mouse(x,y,t) = (2065,2065,1921)
Starting MSE calculation...
Done. Current results = 1462 

Participant:  78652
Loading trials...
Trail 30: mouse(x,y,t) = (2335,2335,1920)
Trail 32: mouse(x,y,t) = (2052,2052,1920)
Trail 36: mouse(x,y,t) = (2014,2014,1920)
Trail 38: mouse(x,y,t) = (2015,2015,1920)
Trail 43: mouse(x,y,t) = (2012,2012,1920)
Trail 48: mouse(x,y,t) = (2016,2016,1920)
Starting MSE calculation...
Done. Current results = 1498 

Participant:  78992
Loading trials...
Trail 3: mouse(x,y,t) = (2746,2746,1919)
Trail 5: mouse(x,y,t) = (2109,2109,1920)
Trail 7: mouse(x,y,t) = (2146,2146,1921)
Trail 12: mouse(x,y,t) = (2183,2183,1921)
Trail 18: mouse(x,y,t) = (2205,2205,1921)
Trail 22: mouse(x,y,t) = (2022,2022,1918)
Starting MSE calculation...
Done. Current results = 1534 

Participant:  86901
Loading trials..

In [15]:
result.to_csv('probe_results.csv')

In [16]:
for file_name in glob.glob(r"C:\Users\Robyn\Downloads\!experiment\data\WhatsApp"+'\*.csv'):
    participant_nr = file_name[47+4:52+4]
    print('Participant: ', participant_nr)
    data = pd.read_csv('{}'.format(file_name))
    data = data.replace(np.nan, '')
    probeCaught(data, participant_nr)
    print('Done. Current results =', len(result), '\n')

Participant:  10739
Loading trials...
Trail 6: mouse(x,y,t) = (2184,2184,1923)
Trail 9: mouse(x,y,t) = (2130,2130,1923)
Trail 11: mouse(x,y,t) = (2143,2143,1923)
Trail 14: mouse(x,y,t) = (2048,2048,1922)
Trail 19: mouse(x,y,t) = (2075,2075,1923)
Starting MSE calculation...
Done. Current results = 1842 

Participant:  14786
Loading trials...
Trail 6: mouse(x,y,t) = (2314,2314,1921)
Trail 9: mouse(x,y,t) = (2213,2213,1921)
Trail 15: mouse(x,y,t) = (2209,2209,1921)
Trail 17: mouse(x,y,t) = (2075,2075,1921)
Trail 19: mouse(x,y,t) = (2224,2224,1921)
Trail 23: mouse(x,y,t) = (2223,2223,1921)
Starting MSE calculation...
Done. Current results = 1878 

Participant:  33349
Loading trials...
Trail 30: mouse(x,y,t) = (2190,2190,1926)
Trail 33: mouse(x,y,t) = (2245,2245,1926)
Trail 38: mouse(x,y,t) = (2353,2353,1927)
Trail 40: mouse(x,y,t) = (2358,2358,1926)
Trail 44: mouse(x,y,t) = (2087,2087,1926)
Trail 50: mouse(x,y,t) = (2093,2093,1926)
Starting MSE calculation...
Done. Current results = 1914 


In [17]:
result.to_csv('probe_results_all.csv')

In [ ]:
f = 0
u = 0
a = 0
space = 0
notspace = 0
count = 0

for file_name in glob.glob(r"C:\Users\Robyn\Downloads\!experiment\data\SONA"+'\*.csv'):
    participant_nr = file_name[47:52]
    df = pd.read_csv('{}'.format(file_name))
    df = df.replace(np.nan, '')
    f += len(df['key_resp_2.keys'][df['key_resp_2.keys']  == '["f"]'].index.tolist())
    u += len(df['key_resp_2.keys'][df['key_resp_2.keys']  == '["a"]'].index.tolist())
    a += len(df['key_resp_2.keys'][df['key_resp_2.keys']  == '["u"]'].index.tolist())#  perform calculation
    space += len(df['selfReport.keys'][df['selfReport.keys']  == '["space"]'].index.tolist())
    notspace += len(df[df['selfReport.keys'] != ''])  
    
    count += 1
print(f,a,u, space, notspace, count) 